In [1]:
# Setting up the tic-tac-toe board/rules that we will use later

import random;
import sys;
import numpy as np;

def findWinner( board ):
    """
        Finds the winner of the board, returns 1, -1 for winner or 0 if no winner
    """
    winning_positions = [
        # horizontal
        [ 0, 1, 2 ], # < Heh, this is actually a 2d representation of the board
        [ 3, 4, 5 ],
        [ 6, 7, 8 ],
        # vertical
        [ 0, 3, 6 ],
        [ 1, 4, 7 ],
        [ 2, 5, 8 ],
        # diagonal
        [ 0, 4, 8 ],
        [ 2, 4, 6 ]
    ]
    for winning_position in winning_positions:
        if( board[ winning_position[ 0 ] ] == board[ winning_position[ 1 ] ] == board[ winning_position[ 2 ] ] != 0 ):
            return board[ winning_position[ 0 ] ]
    return 0

def printBoard( board ):
    """
        Prints out a board prettily. Note: calls findWinner
    """
    assert( len( board ) == 9 )
    for i, spot in enumerate( board ):
        if i != 0:
            if i % 3 == 0:
                sys.stdout.write( '\n--------------\n' )
            else: 
                sys.stdout.write( '|' )
        sys.stdout.write( str( spot ).center( 4, ' ' ) )
    sys.stdout.write( ' winner: ' + str( findWinner( board ) ) )
    sys.stdout.write( '\n' )

def generate():
    """
        Generates a set of random boards
    """
    current_player = random.choice( [ -1, 1 ] )
    board = [ 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
    boards = [ list( board ) ]
    moves = []
    positions = [ 0, 1, 2, 3, 4, 5, 6, 7, 8 ]

    for i in range( 9 ):
        move = positions.pop( random.randint( 0, len( positions ) - 1 ) )
        board[ move ] = current_player
        current_player *= -1
        moves.append( move )
        boards.append( list( board ) )
        winner = findWinner( board )
        if winner:
            return board, boards, moves, winner
    return board, boards, moves, 0
        
printBoard( [ 0, 1, -1, 0, -1, 0, 1, 1, 1 ] ) 

board, boards, moves, winner = generate()

printBoard( board )
print boards
print moves

 0  | 1  | -1 
--------------
 0  | -1 | 0  
--------------
 1  | 1  | 1   winner: 1
 1  | 1  | 0  
--------------
 -1 | -1 | -1 
--------------
 1  | 1  | -1  winner: -1
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, -1, 0, 0, 0, 0], [1, 0, 0, 0, -1, 0, 1, 0, 0], [1, 0, 0, -1, -1, 0, 1, 0, 0], [1, 1, 0, -1, -1, 0, 1, 0, 0], [1, 1, 0, -1, -1, 0, 1, 0, -1], [1, 1, 0, -1, -1, 0, 1, 1, -1], [1, 1, 0, -1, -1, -1, 1, 1, -1]]
[0, 4, 6, 3, 1, 8, 7, 5]


In [2]:
# Testing out how the neural net is going to work

def softmax( x ):
    """Compute softmax values for each sets of scores in x."""
    return np.exp( x ) / np.sum( np.exp( x ), axis=0 )

test_input = np.full( ( 1, 10 ), 0 )
test_input[ 0, 0 ] = 1 # Biases: just a 1 (otherwise the first move will always be random)

# 11 node hidden layer
hidden_weights = np.random.randn( 10, 11 )
hidden_biases = np.full( ( 11 ), 1 )

# hidden values
hidden_values = np.dot( test_input, hidden_weights ) + hidden_biases

# outputs layer
output_weights = np.random.randn( 11, 9 )
output_biases = np.full( ( 9 ), 1 )

# output values
output_values = np.dot( hidden_values, output_weights ) + output_biases
print( output_values )
output_values = softmax( output_values[ 0 ] )
print( output_values )

[[  6.1804291   -0.92841175  -1.94035023   1.63628233   2.27472248
    1.67639636   2.89501517  10.73639247  -0.28850598]]
[  1.03863371e-02   8.49438731e-06   3.08782558e-06   1.10398849e-04
   2.09042977e-04   1.14917414e-04   3.88709604e-04   9.88762904e-01
   1.61079251e-05]


In [3]:
def softmax( x ):
    """Compute softmax values for each sets of scores in x."""
    return np.exp( x ) / np.sum( np.exp( x ), axis=0 )

class Player:
    """
        A tic-tac-toe player, responsible for using it's weights to
        choose a move
        
        If you don't specify weights it will generate it's own random ones
        
        Has a simple one hidden layer NN and biases

        Eventually we will make player more generic and have configurable parameters so we can play
        around, but that's for a different project
    """
    def __init__( self, weights=[] ):
        if len( weights ) == 0:
            weights = np.array( [
                np.random.randn( 10, 11 ),
                np.random.randn( 11, 9 )
            ] )

        # Am I doing this right?
        biases = np.array( [
            np.full( ( 11 ), 1 ),
            np.full( ( 9 ), 1 )
        ] )
        
        layers = []
        for i, val in enumerate( weights ):
            layers.append( [ weights[ i ], biases[ i ] ] )
        self.layers = np.array( layers )
            
    def getHighestScoredValidMove( self, current_board, output_values ):
        for move_choice in output_values.argsort()[ ::-1 ]:
            if current_board[ move_choice ] == 0:
                return move_choice;

    def performLayer( self, previous_values, current_layer ):
        current_layer_values = np.dot( previous_values, current_layer[ 0 ] )
        current_layer_values += current_layer[ 1 ]
        return current_layer_values
            
    def getMove( self, current_board ):
        # Create the input layer from the board
        input_layer = np.full( ( 1, 10 ), 0 )
        input_layer[ 0 ][ 0 ] = 1
        input_layer[ 0 ][ 1: ] = current_board

        output_values = input_layer
        for layer in self.layers:
            output_values = self.performLayer( output_values, layer )
        output_values = softmax( output_values[ 0 ] )

        return self.getHighestScoredValidMove( current_board, output_values )

class RandomPlayer:
    def getMove( self, current_board ):
        choices = []
        for i, val in enumerate( current_board ):
            if current_board[ i ] == 0:
                choices.append( i )
        return random.choice( choices )
    
player_1 = Player()
player_1.getMove( [ 0, 0, 0, 1, 0, 0, 0, 0, 0 ] ) 

random_player_test = RandomPlayer()
random_player_test.getMove( [ 0, 0, 0, 0, 0, 0, 0, 0, 0 ] )

weights = [
    np.random.randn( 10, 11 ),
    np.full( ( 11, 9 ), 0 )
]
weights[ 1 ][:,3] = 1
deterministic_player = Player( weights )
determined_move = deterministic_player.getMove( [ 0, 0, 0, 0, 0, 0, 0, 0, 0 ] ) 
assert( determined_move == 3 )

player = Player()
only_move = player_1.getMove( [ 0, 1, 1, 1, 1, 1, 1, 1, 1 ] ) 
assert( only_move == 0 )

In [4]:
class Board:
    """
        Board is a class that takes in a set (2) of players, and is responsible for
        pitting them against eachother
    """
    def __init__( self, players ):
        assert( len( players ) == 2 )

        self.players = players
        self.reset()
        
    def convertBoard( self, is_first_player ):
        if is_first_player:
            return self.board
        return self.board * -1

    def getMove( self, current_player_num, is_first_player ):
        converted_board = self.convertBoard( is_first_player )
        return self.players[ current_player_num ].getMove( converted_board )

    def play( self ):
        """
            Plays a single game of tic-tac-toe
            Returns
                winner (index in the player array of the winner)
                turn (how many turns it took for the game to end)
                first_player (index in the player array of the first-player)
        """
        self.reset()
        # Choose a player to go first
        first_player = random.randint( 0, 1 )
        current_player_num = first_player

        turn = 0
        winner = 0

        while turn < 9 and winner == 0:
            move = self.getMove( current_player_num, ( current_player_num == first_player ) )
            assert( self.board[ move ] == 0 )
            self.board[ move ] = 1 if current_player_num == first_player else -1
            current_player_num = ( current_player_num + 1 ) % 2
            turn += 1
            winner = findWinner( self.board )

        converted_winner = first_player if winner == 1 else ( first_player + 1 ) % 2
        return converted_winner, turn - 1, first_player

    def reset( self ):
        self.board = np.array( [ 0, 0, 0, 0, 0, 0, 0, 0, 0 ] )
        
player_1 = Player()
player_2 = Player()
# Test the board
board = Board( [ player_1, player_2 ] )
for i in range( 2 ):
    print( board.play() )

(1, 6, 1)
(1, 7, 0)


In [20]:
validation_players = [RandomPlayer() for i in range( 100 )]

variance = .01
random_addition = 1 - variance / 2

def averageWeights( weights_1, weights_2 ):
    """
        Very simple reproduction algorithm. Takes the weights of the
        parents and averages them together, then applies some random
        variation to the values after
        
        child = ( ( parent_1 + parent_2 ) / 2 ) * ( random_matrix * variance + random_addition )
    """
    assert( len( weights_1 ) == len( weights_2 ) )

    new_weights = []
    for i, val in enumerate( weights_1 ):
        new_layer = np.mean( np.array( [ weights_1[ i ], weights_2[ i ] ] ), axis=0 )
        new_layer *= ( np.random.random( new_layer.shape ) * variance + random_addition )
        new_weights.append( new_layer )
    return new_weights

class Generation:
    """
        The generation class is responsible for taking in a list of players, 
        playing them against eachother, choosing the best, and returning a list
        of players making up of the next generation
        
        Also allows you to compare the data to the validation players
    """

    def __init__( self, players ):
        self.players = players

    def compareToRandom( self, games ):
        """
            Plays the current generation against the validation set for a given number of games
            and returns the results
            
            Output
            [
                [wins_going_first, games_going_first],
                [wins_going_second, games_going_second]
            ]
        """
        wins = np.full( ( 2, 2 ), 0 )

        for i in range( games ):
            player = random.choice( self.players )
            random_player = random.choice( validation_players )

            board = Board( [ random_player, player ] )

            results = board.play()
            if results[ 0 ] == 1:
                if results[ 2 ] == 1:
                    wins[ 0 ][ 0 ] += 1
                else:
                    wins[ 1 ][ 0 ] += 1
            if results[ 2 ] == 1:
                wins[ 0 ][ 1 ] += 1
            else:
                wins[ 1 ][ 1 ] += 1
        return wins
        
    def runAndReproduce( self ):
        results = np.full( 100, 0 )

        for i in range( 2000 ):
            # Chose two players
            player_1_i = player_2_i = random.randint( 0, 99 )
            while player_2_i == player_1_i:
                player_2_i = random.randint( 0, 99 )
            assert( player_1_i != player_2_i )

            chosen_players_i = [ player_1_i, player_2_i ]

            board = Board( [ self.players[ player_1_i ], self.players[ player_2_i ] ] )

            # Play 5 games
            for i in range( 5 ):
                board_results = board.play()
                winner_i = chosen_players_i[ board_results[ 0 ] ]
                results[ winner_i ] += 1

        # Choose the best of the previous generation
        sorted_results = results.argsort()

        top_30 = sorted_results[ ::-1 ][ 0 : 30 ]
        top_30_values = map( lambda i: results[ i ], top_30 )

        new_generation = []

        # Create a new generation
        for i in range( 100 ):
            top_30_player_1_i = top_30_player_2_i = random.randint( 0, 29 )
            while top_30_player_2_i == top_30_player_1_i:
                top_30_player_2_i = random.randint( 0, 29 )
            player_1 = self.players[ top_30[ top_30_player_1_i ] ]
            player_2 = self.players[ top_30[ top_30_player_2_i ] ]

            new_weights = averageWeights( player_1.layers[:,0], player_2.layers[:,0] )

            new_generation.append( Player( new_weights ) )

        return new_generation

In [23]:
players = []
for i in range( 100 ):
    players.append( Player() )

for i in range( 1, 31 ):
    generation = Generation( players )
    validation = generation.compareToRandom( 500 )
    
    winrate = np.sum( validation[ :, 0 ] ) / 500.
    turn_order_winrates = validation[ :, 0 ].astype( float ) / validation[ :, 1 ].astype( float )
    
    output = 'Generation: ' + str( i ).rjust( 2, ' ' ) + ' ' + \
         '    Winrate: %.4f' % winrate + \
         '    Winrate going first: %4f' % turn_order_winrates[ 0 ] + \
         '    Winrate going second: %4f' % turn_order_winrates[ 1 ]

    print( output )

    players = generation.runAndReproduce()

Generation:  1     Winrate: 0.5380    Winrate going first: 0.682819    Winrate going second: 0.417582
Generation:  2     Winrate: 0.5820    Winrate going first: 0.702041    Winrate going second: 0.466667
Generation:  3     Winrate: 0.6560    Winrate going first: 0.738589    Winrate going second: 0.579151
Generation:  4     Winrate: 0.6760    Winrate going first: 0.760331    Winrate going second: 0.596899
Generation:  5     Winrate: 0.6900    Winrate going first: 0.826923    Winrate going second: 0.541667
Generation:  6     Winrate: 0.7600    Winrate going first: 0.836207    Winrate going second: 0.694030
Generation:  7     Winrate: 0.7140    Winrate going first: 0.782288    Winrate going second: 0.633188
Generation:  8     Winrate: 0.7140    Winrate going first: 0.778598    Winrate going second: 0.637555
Generation:  9     Winrate: 0.6640    Winrate going first: 0.748899    Winrate going second: 0.593407
Generation: 10     Winrate: 0.7060    Winrate going first: 0.803419    Winrate goi

### Next steps

So this very very basic algorithm seems to have achieve mildly good results, especially for the game of tic-tac-toe.

There are a ton of things that I want to try to implement using this really simple base, but I think first I need to strengthen the base up. Most of the pieces are implemented in classes/methods, but most of those methods don't take input, so it is hard to mess with hyperparmeters. So __first__ I am going to solidify the base I am working with

Things I want to try after that:

 - Create a smarter fitness function. Right now it is optimizing for *wins* which is alright for this simple model, but there are other interesting things that I want to try
     - Like optimizing for winning when we go first, and not-losing when going second. This current fitness function doesn't even take into account ties
 - Build something that will try out a whole bunch of hyperparamters at once, so we don't have to manually tweak and
 run, and it is much easier to compare
     - Because of the nature of genetic algorithms like this, it should
 - Play around with other ways to do reproduction. 
     - Maybe it could be a weighted average of a set of parents, rather than the really simple average of two parents I am using here.
     - Or a weighted average of all of the top_30
 - Give the NN some memory. So the end decision will be made by our geneticly growing neural net, but there will be another set of inputs that comes from a FNN that tries to score every position for it's likelyhood to win us this game. The reproduction step will be 1. Perform gradient descent on the memory, 2. Combine and mutate the parents. This will act as a sort of historical memory of all games played